# Custom Chatbot Project

## Scenario
Because of the llm training date information regarding an actual bmw is difficult to get from an llm and sometimes it hallucinating some information. 
instead of connecting to an internal database which i can't use for this project, i decided to scrap the us bmw website to get some actual information
for 3 cars and feed this information into a prompt.


## Data collection

In [338]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import openai
import os
openai.api_key = os.getenv('OPENAI_API_KEY')
#DataPath
path = "./data/"


In [252]:
# Function to fetch a page
def fetch_page(url: str):
    headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    r = requests.get(url)
    if r.status_code == 200:
        return r.text
    else:
        print(r.status_code)
        return r.text

In [291]:
#research_url = https://www.bmwusa.com/vehicles/all-electric/i5/sedan/electric-bmw-m.html
#research_url2 = https://www.bmwusa.com/vehicles/all-electric/ix/suv/electric-bmw-m.html

# Get the source of the page
#source = fetch_page(research_url)

# Write the source to disk
#with open("https://www.bmwusa.com/vehicles/all-electric/i5/sedan/electric-bmw-m.html", "w") as f:
#    f.write(source)
#had to store the pages localy because of access issues
    
# Load the source from disk
with open(path + "2025BMWiXxDrive50All-ElectricSUV_ Features&Specs.html", "r") as f:
    source = f.read()
with open(path + "2025BMWiXxDrive50All-ElectricSUV_ Features&Specs.html", "r") as f:
    source2 = f.read()
with open(path + "BMWi5M60All-ElectricSedan_BMWUSA.html", "r") as f:
    source3 = f.read()

In [369]:
def parsePage2(source : str, vehicle_type : str) -> list: #As far as i understand the Text is placed
    soup = BeautifulSoup(source, 'html.parser')
    
    #e.g. Performance and efficieny
    headers = soup.find_all('div', { 'class' : "category-container--section-wrapper"})
    #print(len(headers))
    featureList : list = []
    for header in headers:
        #Idetify the headline
        features_Headline = header.find('h4', { 'class' : "section-header__copy h-left headline-6--bold"})
        if(features_Headline != None):
            #print(f'Headline : {features_Headline.text}')
            text : str = features_Headline.text
            #removing leading and  trainling whitespaces
            text = vehicle_type + " " + text.strip() + ': '
        #identify the features
        features = header.find_all('div', { 'class' : "body-text__copy body-copy text-content content-2 width-l"})
        for feature in features:
            #print(f'\t\t{ feature.text}')
            text += feature.text.strip()
        featureList.append(text)
    return featureList
featureList = parsePage2(source, "iX xDrive50")

In [370]:
len(featureList)

17

In [371]:
import pandas as pd
df_1 = pd.DataFrame(featureList, columns=['text'])
df_1

,text
0,iX xDrive50 Performance and Efficiency: Dual a...
1,"iX xDrive50 Handling, Ride and Braking: xDrive..."
2,iX xDrive50 Exterior: Aluminum Spaceframe with...
3,iX xDrive50 Interior Trim: Perforated Sensatec...
4,iX xDrive50 Connectivity: Connected Package Pr...
5,iX xDrive50 Audio System: SiriusXM® with 360L ...
6,iX xDrive50 Instrumentation and Controls: Mult...
7,iX xDrive50 Comfort and Convenience: Parking A...
8,iX xDrive50 Safety and Security: Mobility kitA...
9,iX xDrive50 Warranty: BMW Ultimate Care (for c...


In [382]:
featureList = parsePage2(source2, "iX M60 SUV")
df_2 = pd.DataFrame(featureList, columns=['text'])

#df_all = pd.concat([df, pd.DataFrame(featureList, columns=['text'])], ignore_index=True)
featureList = parsePage2(source3, "i5 M60 All-Electric Sedan")
df_3 = pd.DataFrame(featureList, columns=['text'])
#df_all = pd.concat([df_all, pd.DataFrame(featureList, columns=['text'])], ignore_index=True)
df_all = pd.concat([df_1, df_2, df_3])

df_all = df_all.reset_index(drop=True)
df_all.to_csv(path + 'bmw_webcontent.csv', index=True, index_label='index')


In [383]:

df_all

,text
0,iX xDrive50 Performance and Efficiency: Dual a...
1,"iX xDrive50 Handling, Ride and Braking: xDrive..."
2,iX xDrive50 Exterior: Aluminum Spaceframe with...
3,iX xDrive50 Interior Trim: Perforated Sensatec...
4,iX xDrive50 Connectivity: Connected Package Pr...
5,iX xDrive50 Audio System: SiriusXM® with 360L ...
6,iX xDrive50 Instrumentation and Controls: Mult...
7,iX xDrive50 Comfort and Convenience: Parking A...
8,iX xDrive50 Safety and Security: Mobility kitA...
9,iX xDrive50 Warranty: BMW Ultimate Care (for c...


## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [403]:
import pandas as pd

df = pd.read_csv(path + 'bmw_webcontent.csv', index_col=['index'])
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, 0 to 50
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    51 non-null     object
dtypes: object(1)
memory usage: 816.0+ bytes


In [404]:
#Check for duplicates
df.drop_duplicates(['text'], keep='first', inplace=True)

In [405]:
df

,text
index,
0,iX xDrive50 Performance and Efficiency: Dual a...
1,"iX xDrive50 Handling, Ride and Braking: xDrive..."
2,iX xDrive50 Exterior: Aluminum Spaceframe with...
3,iX xDrive50 Interior Trim: Perforated Sensatec...
4,iX xDrive50 Connectivity: Connected Package Pr...
5,iX xDrive50 Audio System: SiriusXM® with 360L ...
6,iX xDrive50 Instrumentation and Controls: Mult...
7,iX xDrive50 Comfort and Convenience: Parking A...
8,iX xDrive50 Safety and Security: Mobility kitA...


In [410]:
#Replace \n in all data for the openai api
df = df.replace('\n', ' ', regex=True)
df = df.replace('(for complete details, visit click here)', ' ', regex=True)
df = df.replace(r'\(\)', ' ', regex=True)
df = df.replace(r'\( \)', ' ', regex=True)


In [412]:
#Set new index
df = df.reset_index(drop=True)


In [414]:
#Convert the content into rag vectors via Open API
#Tokenizer for gpt-3
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 100
embeddings = []
for i in range(0, len(df), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = openai.Embedding.create(
        input=df.iloc[i:i+batch_size]["text"].tolist(),
        engine=EMBEDDING_MODEL_NAME
    )
    
    # Add embeddings to list
    embeddings.extend([data["embedding"] for data in response["data"]])

# Add embeddings list to dataframe
df["embeddings"] = embeddings
df

,text,embeddings
0,iX xDrive50 Performance and Efficiency: Dual a...,"[0.0035003223456442356, 0.00087758555309847, -..."
1,"iX xDrive50 Handling, Ride and Braking: xDrive...","[-0.010590382851660252, 0.006510048173367977, ..."
2,iX xDrive50 Exterior: Aluminum Spaceframe with...,"[0.016707586124539375, -0.0007189466850832105,..."
3,iX xDrive50 Interior Trim: Perforated Sensatec...,"[-0.009998289868235588, 0.0020799601916223764,..."
4,iX xDrive50 Connectivity: Connected Package Pr...,"[-0.007763830479234457, -0.0010993833420798182..."
5,iX xDrive50 Audio System: SiriusXM® with 360L ...,"[0.0017285030335187912, -0.0038443186786025763..."
6,iX xDrive50 Instrumentation and Controls: Mult...,"[-0.019040042534470558, -0.015499262139201164,..."
7,iX xDrive50 Comfort and Convenience: Parking A...,"[0.0038548761513084173, 0.0005722081987187266,..."
8,iX xDrive50 Safety and Security: Mobility kitA...,"[-0.0017678706208243966, -0.009593148715794086..."
9,iX xDrive50 Warranty: BMW Ultimate Care 4-yea...,"[0.006260425318032503, -0.016591470688581467, ..."


In [415]:
df.to_csv(path + "embeddings.csv")

In [ ]:
#For a fresh start
# import numpy as np
# import pandas as pd
# import openai
# openai.api_key = os.getenv('OPENAI_API_KEY')
# df = pd.read_csv("embeddings.csv", index_col=0)
# df["embeddings"] = df["embeddings"].apply(eval).apply(np.array)

## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [416]:
from openai.embeddings_utils import get_embedding, distances_from_embeddings

def get_rows_sorted_by_relevance(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """
    
    # Get embeddings for the question text
    question_embeddings = get_embedding(question, engine=EMBEDDING_MODEL_NAME)
    
    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )
    
    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy

In [421]:
get_rows_sorted_by_relevance("total power output of the M60", df)

,text,embeddings,distances
10,iX M60 SUV Performance and Efficiency: Dual al...,"[0.00986961554735899, 0.008943709544837475, 0....",0.170995
20,i5 M60 All-Electric Sedan Performance and Effi...,"[-0.0051428284496068954, 0.01057688519358635, ...",0.182865
22,"i5 M60 All-Electric Sedan Exterior: 19"" M Aero...","[-0.006551796570420265, 0.02140757627785206, 0...",0.214510
21,"i5 M60 All-Electric Sedan Handling, Ride and B...","[-0.01075008325278759, 0.022665200755000114, 0...",0.217884
15,iX M60 SUV Audio System: SiriusXM® with 360L +...,"[0.01137599814683199, 0.0025020099710673094, -...",0.219271
11,"iX M60 SUV Handling, Ride and Braking: xDrive ...","[-0.006685336586087942, 0.011505152098834515, ...",0.220007
16,iX M60 SUV Instrumentation and Controls: Multi...,"[-0.010806947946548462, -0.00921907089650631, ...",0.223001
0,iX xDrive50 Performance and Efficiency: Dual a...,"[0.0035003223456442356, 0.00087758555309847, -...",0.224701
27,i5 M60 All-Electric Sedan Comfort and Convenie...,"[0.01158122718334198, -0.0002572140365373343, ...",0.225096
17,iX M60 SUV Comfort and Convenience: Parking As...,"[0.01092325896024704, 0.006389837246388197, 0....",0.226589


## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

### Question 2